# Validating a Linear Regression

[Download the Excel file here](https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls) on crime data in New York State in 2013, provided by the FBI: UCR ([Thinkful mirror](https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv)).

Prepare this data to model with multivariable regression (including data cleaning if necessary) according to this specification:

$$ Property crime = \alpha + Population + Population^2 + Murder + Robbery$$

Features identified through analysis [here](https://github.com/ephs08kmp/Thinkful_Bootcamp_Challenges/blob/master/2.4.4%20Challenge%20Multivariable%20Regression%20Model.ipynb). Code below to set features for modeling and validation.

## Preparing the Data

In [2]:
#necessary imports
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn import metrics
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action='ignore', module='scipy', message='^internal gelsd')

In [3]:
#read in data
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls', header=4)

In [4]:
#calculating median, standard deviation and the limit for outliers
median = df.Population.median()
std = df.Population.std()
pop_out = median + 2* std
#turning outliers into None
df['Population'] = df.Population.map(lambda x: x if x < pop_out else None)

median = df['Murder and\nnonnegligent\nmanslaughter'].median()
std = df['Murder and\nnonnegligent\nmanslaughter'].std()
mur_out = median + 2* std
df['Murder and\nnonnegligent\nmanslaughter'] = df['Murder and\nnonnegligent\nmanslaughter'].map(lambda x: x if x < mur_out else None)

median = df.Robbery.median()
std = df.Robbery.std()
rob_out = median + 2* std
df['Robbery'] = df.Robbery.map(lambda x: x if x < rob_out else None)

median = df['Property\ncrime'].median()
std = df['Property\ncrime'].std()
prop_out = median + 2* std
df['Property_crime'] = df['Property\ncrime'].map(lambda x: x if x < prop_out else None)

In [5]:
#creating population squared feature
df['PopSq'] = df['Population']**2

In [6]:
#turning murder and robbery into categorical features
df['Murder_cat'] = df['Murder and\nnonnegligent\nmanslaughter'].dropna().map(lambda x: 1 if x > 0 else 0)
df['Robbery_cat'] = df.Robbery.dropna().map(lambda x: 1 if x > 0 else 0)

In [7]:
#isolating data to be used for modeling
data = df[['Population', 'PopSq', 'Murder_cat', 'Robbery_cat', 'Property_crime']].dropna()
data.head()

,Population,PopSq,Murder_cat,Robbery_cat,Property_crime
0,1861.000,3463321.000,0.000,0.000,12.000
1,2577.000,6640929.000,0.000,0.000,24.000
2,2846.000,8099716.000,0.000,0.000,16.000
3,97956.000,9595377936.000,1.000,1.000,4090.000
4,6388.000,40806544.000,0.000,1.000,223.000


## Initial Model - Simple Linear Regression

In [8]:
#Instantiate and fit the model
regr = linear_model.LinearRegression()
Y = data['Property_crime']
X = data[['Population', 'PopSq', 'Murder_cat', 'Robbery_cat']]
regr.fit(X, Y)

#inspect the results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared: \n')
print(regr.score(X, Y))


Coefficients: 
 [ 2.62320223e-02 -3.30058846e-08  1.70080551e+02  1.27606800e+01]

Intercept: 
 -70.88477571593813

R-squared: 

0.7135336089794011


This R-squared value of 0.714 is not bad as a first pass of our model.

## Cross Validation

In [11]:
from sklearn import cross_validation
from sklearn import model_selection
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std


#Splitting data into train and test
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=0.25, random_state=222)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
accuracy = regr.score(X_test, y_test)
print('Accuracy of Test Data:', accuracy)

#cross validation on train data
scores = model_selection.cross_val_score(regr, X_train, y_train, cv=3)

print('Cross Validation\nScores:', scores)
print('Mean:', scores.mean())
print('Standard deviation:', scores.std())

RMSE_test = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('RMSE Test Data:', RMSE_test)

Accuracy of Test Data: 0.604408353934395
Cross Validation
Scores: [0.72916326 0.59135816 0.84034251]
Mean: 0.7202879784789494
Standard deviation: 0.10184098861197934
RMSE Test Data: 547.8202818329678


Higher mean R-squared value for cross validation than test data.

## Ordinary Least Squares Regression

In [12]:
linear_formula = 'Property_crime ~ Population+PopSq+Murder_cat+Robbery_cat'

lm = smf.ols(formula=linear_formula, data=data).fit()

print('Coefficients:\n', lm.params)
print('\nP-values:\n', lm.pvalues)
print('\nR-squared:\n', lm.rsquared)
lm.conf_int()

Coefficients:
 Intercept     -70.885
Population      0.026
PopSq          -0.000
Murder_cat    170.081
Robbery_cat    12.761
dtype: float64

P-values:
 Intercept     0.016
Population    0.000
PopSq         0.019
Murder_cat    0.006
Robbery_cat   0.761
dtype: float64

R-squared:
 0.7135336089794011


,0,1
Intercept,-128.429,-13.340
Population,0.022,0.030
PopSq,-0.000,-0.000
Murder_cat,48.925,291.236
Robbery_cat,-69.581,95.102


Based on the model, we see that Population squared  has a coefficient of zero, and has a p-value of 0.000 as well. We should eliminate this feature from the model.

##### OLS Iteration 2

In [13]:
linear_formula2 = 'Property_crime ~ Population+Murder_cat+Robbery_cat'

lm2 = smf.ols(formula=linear_formula2, data=data).fit()
print('Coefficients:\n', lm2.params)
print('\nP-values:\n', lm2.pvalues)
print('\nR-squared:\n', lm2.rsquared)
lm2.conf_int()

Coefficients:
 Intercept     -53.405
Population      0.022
Murder_cat    192.486
Robbery_cat    44.234
dtype: float64

P-values:
 Intercept     0.062
Population    0.000
Murder_cat    0.002
Robbery_cat   0.269
dtype: float64

R-squared:
 0.7088744678562764


,0,1
Intercept,-109.429,2.619
Population,0.020,0.024
Murder_cat,71.997,312.974
Robbery_cat,-34.301,122.769


Removing Population Squared left the R-squared unchanged while reducing the p-values to below 0.001. 

##### OLS Version 3

In [35]:
linear_formula3 = 'Property_crime ~ Population+Murder_cat'

lm3 = smf.ols(formula=linear_formula3, data=data).fit()
print('Coefficients:\n', lm3.params)
print('\nP-values:\n', lm3.pvalues)
print('\nR-squared:\n', lm3.rsquared)
lm3.conf_int()

Coefficients:
 Intercept    -32.989
Population     0.022
Murder_cat   201.795
dtype: float64

P-values:
 Intercept    0.130
Population   0.000
Murder_cat   0.001
dtype: float64

R-squared:
 0.707826613002188


,0,1
Intercept,-75.720,9.742
Population,0.020,0.024
Murder_cat,82.407,321.183


### Linear Model Without PopSq

In [36]:
#Instantiate and fit the model
regr2 = linear_model.LinearRegression()
Y2 = data['Property_crime']
X2 = data[['Population', 'Murder_cat', 'Robbery_cat']]
regr2.fit(X2, Y2)

#inspect the results
print('Coefficients: \n', list(zip(X2.columns, regr2.coef_)))
print('\nIntercept: \n', regr2.intercept_)
print('\nR-squared: \n', regr2.score(X2, Y2))

Coefficients: 
 [('Population', 0.02205613985643152), ('Murder_cat', 192.48583974233387), ('Robbery_cat', 44.234313317275586)]

Intercept: 
 -53.40501615830658

R-squared: 
 0.7088744678562764


In [41]:
#Splitting data into train and test
X2_train, X2_test, y2_train, y2_test = cross_validation.train_test_split(X2, Y2, test_size=0.25, random_state=222)
regr2.fit(X2_train, y2_train)
y2_pred = regr2.predict(X2_test)
accuracy2 = regr2.score(X2_test, y2_test)
print('Accuracy of Test Data:', accuracy2)

#cross validation on train data
scores2 = model_selection.cross_val_score(regr2, X2_train, y2_train, cv=3)

print('\nCross Validation\nScores:', scores2)
print('\nMean:', scores2.mean())
print('\nStandard deviation:', scores2.std())
RMSE_test2 = np.sqrt(metrics.mean_squared_error(y2_test, y2_pred))
print('RMSE Test Data:', RMSE_test2)

Accuracy of Test Data: 0.6335879299700926

Cross Validation
Scores: [0.6506852  0.63870207 0.8001555 ]

Mean: 0.6965142573216911

Standard deviation: 0.0734485277222746
RMSE Test Data: 527.2291777163994


#### Linear Regression without PopSq and Robberies

In [37]:
#Instantiate and fit the model
regr3 = linear_model.LinearRegression()
Y3 = data['Property_crime']
X3 = data[['Population', 'Murder_cat']]
regr3.fit(X3, Y3)

#inspect the results
print('Coefficients: \n', list(zip(X3.columns, regr3.coef_)))
print('\nIntercept: \n', regr3.intercept_)
print('\nR-squared: \n', regr3.score(X3, Y3))

Coefficients: 
 [('Population', 0.02237984425144919), ('Murder_cat', 201.79502627668543)]

Intercept: 
 -32.98868160699192

R-squared: 
 0.7078266130021877


In [40]:
#Splitting data into train and test
X3_train, X3_test, y3_train, y3_test = cross_validation.train_test_split(X3, Y3, test_size=0.25, random_state=222)
regr3.fit(X3_train, y3_train)
y3_pred = regr3.predict(X3_test)
accuracy3 = regr3.score(X3_test, y3_test)
print('Accuracy of Test Data:', accuracy3)

#cross validation on train data
scores3 = model_selection.cross_val_score(regr3, X3_train, y3_train, cv=3)

print('\nCross Validation\nScores:', scores3)
print('\nMean:', scores3.mean())
print('\nStandard deviation:', scores3.std())
RMSE_test3 = np.sqrt(metrics.mean_squared_error(y3_test, y3_pred))
print('RMSE Test Data:', RMSE_test3)

Accuracy of Test Data: 0.644313199512648

Cross Validation
Scores: [0.63499857 0.63392671 0.7982762 ]

Mean: 0.6890671608817147

Standard deviation: 0.07722368962328922
RMSE Test Data: 519.4555916179133


## Validation Using Other States
__Colorado:__

In [18]:
#read in data
df_co = pd.read_excel('table_8_offenses_known_to_law_enforcement_colorado_by_city_2013.xls', header=4)
df_nj = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_jersey_by_city_2013.xls', header=4)
df_fl = pd.read_excel('table_8_offenses_known_to_law_enforcement_florida_by_city_2013.xls', header=4)

In [19]:
#setting categorical features and renaming property crime
df_co['Murder_cat'] = df_co['Murder and\nnonnegligent\nmanslaughter'].dropna().map(lambda x: 1 if x > 0 else 0)
df_co['Robbery_cat'] = df_co.Robbery.dropna().map(lambda x: 1 if x > 0 else 0)
df_co['Property_crime'] = df_co['Property\ncrime']
#initializing data with features
data_co = df_co[['Population', 'Murder_cat', 'Robbery_cat', 'Property_crime']].dropna()
data_co.head()

,Population,Murder_cat,Robbery_cat,Property_crime
0,110792.000,0.000,1.000,2666.000
1,6685.000,0.000,0.000,314.000
2,1565.000,0.000,0.000,2.000
3,343484.000,1.000,1.000,10786.000
4,6336.000,0.000,0.000,140.000


In [47]:
#using second linear regression to predict CO data
y_pred_co = lm3.predict(data_co[['Population', 'Murder_cat', 'Robbery_cat']])

RMSE_co = np.sqrt(metrics.mean_squared_error(data_co['Property_crime'], y_pred_co))
print('RMSE Colorado Data:', RMSE_co)

RMSE Colorado Data: 1224.71791381954


__New Jersey:__

In [24]:
#setting categorical features and renaming property crime
df_nj['Murder_cat'] = df_nj['Murder and\nnonnegligent\nmanslaughter'].dropna().map(lambda x: 1 if x > 0 else 0)
df_nj['Robbery_cat'] = df_nj.Robbery.dropna().map(lambda x: 1 if x > 0 else 0)
df_nj['Property_crime'] = df_nj['Property\ncrime']
#initializing data with features
data_nj = df_nj[['Population', 'Murder_cat', 'Robbery_cat', 'Property_crime']].dropna()
data_nj.head()

,Population,Murder_cat,Robbery_cat,Property_crime
0,18150.000,0.000,1.000,237.000
1,8380.000,0.000,1.000,266.000
2,6712.000,0.000,0.000,37.000
3,493.000,0.000,0.000,39.000
4,1812.000,0.000,0.000,18.000


In [48]:
#using second linear regression to predict NJ data
y_pred_nj = lm3.predict(data_nj[['Population', 'Murder_cat', 'Robbery_cat']])

RMSE_nj = np.sqrt(metrics.mean_squared_error(data_nj['Property_crime'], y_pred_nj))
print('RMSE New Jersey Data:', RMSE_nj)

RMSE New Jersey Data: 294.9388767900218


__Florida:__

In [27]:
#setting categorical features and renaming property crime
df_fl['Murder_cat'] = df_fl['Murder and\nnonnegligent\nmanslaughter'].dropna().map(lambda x: 1 if x > 0 else 0)
df_fl['Robbery_cat'] = df_fl.Robbery.dropna().map(lambda x: 1 if x > 0 else 0)
df_fl['Property_crime'] = df_fl['Property\ncrime']
#initializing data with features
data_fl = df_fl[['Population', 'Murder_cat', 'Robbery_cat', 'Property_crime']].dropna()
data_fl.head()

,Population,Murder_cat,Robbery_cat,Property_crime
0,9338.000,0.000,1.000,166.000
1,42040.000,0.000,1.000,1564.000
2,536.000,0.000,0.000,3.000
3,2269.000,0.000,0.000,35.000
4,45397.000,1.000,1.000,1527.000


In [49]:
#using second linear regression to predict FL data
y_pred_fl = lm3.predict(data_fl[['Population', 'Murder_cat', 'Robbery_cat']])

RMSE_fl = np.sqrt(metrics.mean_squared_error(data_fl['Property_crime'], y_pred_fl))
print('RMSE Florida Data:', RMSE_fl)

RMSE Florida Data: 1563.5519914928868


The root mean square error is smallest when predicting the New Jersey data.  This is likely because of the similarities between New York and New Jersey.  Both Colorado and Florida have three times as much error. 

## Validating Using 2014 Data

In [30]:
#read in data
df_2014 = pd.read_excel('Table_8_Offenses_Known_to_Law_Enforcement_by_New_York_by_City_2014.xls', header=4)

In [31]:
#setting categorical features and renaming property crime
df_2014['Murder_cat'] = df_2014['Murder and\nnonnegligent\nmanslaughter'].dropna().map(lambda x: 1 if x > 0 else 0)
df_2014['Robbery_cat'] = df_2014.Robbery.dropna().map(lambda x: 1 if x > 0 else 0)
df_2014['Property_crime'] = df_2014['Property\ncrime']
#initializing data with features
data_2014 = df_2014[['Population', 'Murder_cat', 'Robbery_cat', 'Property_crime']].dropna()
data_2014.head()

,Population,Murder_cat,Robbery_cat,Property_crime
0,1851.000,0.000,0.000,11.000
1,2568.000,0.000,1.000,49.000
2,820.000,0.000,0.000,1.000
3,2842.000,0.000,0.000,17.000
4,98595.000,1.000,1.000,3888.000


In [50]:
#using second linear regression to predict CO data
y_pred_2014 = lm3.predict(data_2014[['Population', 'Murder_cat', 'Robbery_cat']])

RMSE_2014 = np.sqrt(metrics.mean_squared_error(data_2014['Property_crime'], y_pred_2014))
print('RMSE New York 2014 Data:', RMSE_2014)

RMSE New York 2014 Data: 2862.696147310009


Based on the data, this linear regression model best predicts property crime for the same year.  Data in 2014 had a much higher error than the 2013 data.